# Exploratory Analysis and PoC for creating a sentiment analysis app for the EPL Teams

In [1]:
import tweepy
import json
import pandas as pd
from datetime import datetime
import nltk
import boto3
import s3fs
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
# nltk.download('vader_lexicon') # Need this to run SIA()
# need to install pyarrow

In [2]:
def polarity(row):
    #Does the sentiment analysis stuff
    sia = SIA()
    polarity=sia.polarity_scores(row)
    return polarity['compound']

In [3]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    s3_cred_json=d["s3-access"]
    
##auth using s3fs
fs = s3fs.S3FileSystem(anon=False, key=s3_cred_json['ACCESS_KEY_ID'], secret=s3_cred_json['SECRET_ACCESS_KEY'])

In [4]:
with fs.open('s3://mykeysandstuff/query_list.csv', 'r') as f:
    df_query_list=pd.read_csv(f)
#Read the state from the list
with fs.open('s3://mykeysandstuff/index_state.json', 'r') as f:
    index_state_json = json.load(f)
    index=index_state_json["index_state"]
    
    

index=2
#Rotates the key's depending on the index you're on. So that we don't hit the rate limit.
if index%3==0:
    append_str='_1'
elif index%3==1:
    append_str='_2'
else:
    append_str='_3'   
print(append_str)
# Authenticate to Twitter
auth = tweepy.OAuthHandler(cred_json["CONSUMER_KEY" + append_str], cred_json["CONSUMER_SECRET" + append_str])
auth.set_access_token(cred_json["ACCESS_TOKEN" + append_str], cred_json["ACCESS_TOKEN_SECRET" + append_str])

# Create API object
api = tweepy.API(auth)    

_3


In [9]:
tweets=tweepy.Cursor(api.search, q="#Federer", lang="en", tweet_mode='extended').items(1500)
rows_list=[]
for tweet in tweets:
#     print(tweet)
    dict1 = {}
    dict1['twitter_id']=tweet.id
    dict1['username']=tweet.user.name
    dict1['screenname']=tweet.user.screen_name
    dict1['text']=tweet.full_text
    dict1['created_at']=tweet.created_at
    dict1['retweet_count']=tweet.retweet_count
    dict1['location']=tweet.user.location
    dict1['followers']=tweet.user.followers_count
    dict1['friends']=tweet.user.friends_count
    rows_list.append(dict1)

#output to dataframe and then to snappy.parquet form
dftest=pd.DataFrame(rows_list) 

In [10]:
len(dftest)

711

In [8]:
dftest['polarity']=dftest['text'].apply(polarity)

KeyError: 'text'

In [15]:
#select the row with index of the state
row=df_query_list.loc[index]
print(row)
search_id_list=[]
try:
    for file in fs.listdir('twitterdatalake'):
        file_key=file['Key']
        if ".parquet" in file_key and row['file_name'] in file_key and row['category'] in file_key:
            search_id=file_key.split('-')[1]
            search_id_list.append(search_id)

    if not search_id_list:
        last_since_id=None
    else:
        last_since_id=max(search_id_list)

    print(last_since_id)

    #Get Search Results
    search_results=tweepy.Cursor(api.search,q=row['search'], lang="en",since_id=last_since_id, tweet_mode='extended').items(2000) #since_id neeeded here at some point
    rows_list=[]
    for tweet in search_results:
        dict1 = {}
        dict1['twitter_id']=tweet.id
        dict1['username']=tweet.user.name
        dict1['screenname']=tweet.user.screen_name
        dict1['text']=tweet.full_text
        dict1['created_at']=tweet.created_at
        dict1['retweet_count']=tweet.retweet_count
        dict1['location']=tweet.user.location
        dict1['followers']=tweet.user.followers_count
        dict1['friends']=tweet.user.friends_count
        dict1['search_query']=row['search']
        dict1['file_name']=row['file_name']
        dict1['group_name']=row['group_name']
        rows_list.append(dict1)

    #output to dataframe and then to snappy.parquet form
    df=pd.DataFrame(rows_list) 
    file_dir='s3://twitterdatalake/'
    df['polarity']=df['text'].apply(polarity)
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    max_twitter_id=max(df['twitter_id'])
    full_filename=file_dir+row['category'] +'/'+ row['file_name'] +'/'+  to_csv_timestamp + '-' + str(max_twitter_id)
    with fs.open(full_filename+ '.csv','w', encoding="utf-8", newline = '\n') as f:
        df.to_csv(f)
#     with fs.open(full_filename+ '.snappy.parquet','w', encoding="utf-8") as f:
#         df.to_parquet(f,compression='SNAPPY')


    #Increment the state and make sure it will loop back next round
    index_state_json['index_state']=(index+1)%len(df_query_list)
    print(index_state_json['index_state'])
    with fs.open('s3://mykeysandstuff/index_state.json', 'w') as outfile:
        json.dump(index_state_json, outfile)
        print(index_state_json)
    
except Exception as e:
    print('Exception here:' + str(e))


file_name      WashingtonWizards
group_name     WashingtonWizards
category                     NBA
search        #WashingtonWizards
Name: 126, dtype: object
None
127
{'index_state': 127}


In [5]:




# Create a tweet
api.update_status("Hello Tweepy2")

TweepError: Read-only application cannot POST.

In [11]:
df.to_csv('test.csv', index=False)



In [12]:
df_clean = df.replace('\n',' ', regex=True)

In [14]:
df_clean.to_csv('test1.csv', index=False)

In [148]:
df1 = pd.DataFrame([1])
with fs.open('s3://twitterdatalake/adds.csv','w') as f:
    df1.to_csv(f)

In [156]:
df.to_csv('cscs.csv')

In [ ]:
###Lets tweet this info#####

In [101]:
# average_str='Average: ' + str(df['polarity'].mean().round(2))
# median_str='Median: ' + str(df['polarity'].median().round(2))
# std_str='Standard Deviation: ' + str(df['polarity'].std().round(2))
# row1='Sentiment around: ' + df['file_name'][0]
# row2='Search Query: ' + df['search_query'][0]
# row3=average_str + ', ' + median_str + ', ' +  std_str

# final_str=row1 + '\n'+ row2 + '\n' + row3
# print(final_str)
# # # Create a tweet
# api.update_status(final_str)

Sentiment around: CrystalPalace
Search Query: #CrystalPalace OR #CPFC
Average: 0.35, Median: 0.4, Standard Deviation: 0.3


Status(_api=<tweepy.api.API object at 0x000001938ADCFA58>, _json={'created_at': 'Sat Jul 04 06:35:30 +0000 2020', 'id': 1279302799243661314, 'id_str': '1279302799243661314', 'text': 'Sentiment around: CrystalPalace\nSearch Query: #CrystalPalace OR #CPFC\nAverage: 0.35, Median: 0.4, Standard Deviation: 0.3', 'truncated': False, 'entities': {'hashtags': [{'text': 'CrystalPalace', 'indices': [46, 60]}, {'text': 'CPFC', 'indices': [64, 69]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://en.wikipedia.org/wiki/Uniform_Resource_Locator" rel="nofollow">bumbumapp</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 4073594774, 'id_str': '4073594774', 'name': 'Will', 'screen_name': 'iamwilliamj1', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_

In [90]:
#Most Positive text
df[df['polarity']==max(df['polarity'])]['text'].iloc[0]

'Let’s go! Great to be in contact and thanks @holmesdaleUSA for the intro! “Charleston Palace” has a great ring to i… https://t.co/OA6OrK3UoH'

In [91]:
#Most Negative text
df[df['polarity']==min(df['polarity'])]['text'].iloc[0]

"We've previewed today's #PremierLeague matches here - https://t.co/G2f2e23fCu\n\n#football #betting #tips #EPL… https://t.co/JJnef2CnIp"

In [63]:
df

,twitter_id,username,screenname,text,created_at,retweet_count,location,followers,friends,search_query,file_name,group_name,polarity
0,1279709245425037312,Ángel.,angel_rm13,RT @LastWordOnSpurs: ❓How do you change Spurs ...,2020-07-05 09:30:34,27,España.,159,567,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.0000
1,1279709218186997764,Leeroy,LRC_1978,"Level on points with Burnley, if anyone can ta...",2020-07-05 09:30:28,0,N17,703,595,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.5707
2,1279709165678669829,Adedayo Ben Lanre,lourd4lanre,RT @MrTomMcDermott: The first goal yesterday d...,2020-07-05 09:30:15,5,"Akure,Nigeria.",1688,3648,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.1027
3,1279709105968549888,Luke Conlaund,LukeConlaund4,RT @LastWordOnSpurs: ❓How do you change Spurs ...,2020-07-05 09:30:01,27,,210,1224,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.0000
4,1279709079015960577,Luke Conlaund,LukeConlaund4,RT @LastWordOnSpurs: 🇫🇷 Tottenham Hotspur midf...,2020-07-05 09:29:54,27,,210,1224,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.7184
5,1279708986191855617,Last Word On Spurs🎙,LastWordOnSpurs,RT @LastWordOnSpurs: 🇵🇹 Jose Mourinho doesn’t ...,2020-07-05 09:29:32,9,United Kingdom,39098,398,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,-0.5574
6,1279708955233697792,Last Word On Spurs🎙,LastWordOnSpurs,RT @LastWordOnSpurs: 🇫🇷 Tottenham Hotspur midf...,2020-07-05 09:29:25,27,United Kingdom,39098,398,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.7184
7,1279708915417178112,Last Word On Spurs🎙,LastWordOnSpurs,RT @LastWordOnSpurs: ❓How do you change Spurs ...,2020-07-05 09:29:15,27,United Kingdom,39098,398,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,0.0000
8,1279708877551001600,Last Word On Spurs🎙,LastWordOnSpurs,RT @LastWordOnSpurs: 🇳🇱 Former Tottenham Hotsp...,2020-07-05 09:29:06,127,United Kingdom,39098,398,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,-0.4588
9,1279708854390001666,Last Word On Spurs🎙,LastWordOnSpurs,RT @LastWordOnSpurs: ⚪️ Nigel de Jong blames t...,2020-07-05 09:29:01,185,United Kingdom,39098,398,#Tottenham OR #THFC OR #SPURS,TottenhamHotspur,Tottenham Hotspur FC,-0.7003
